In [1]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
# %pip install understatapi
from understatapi import UnderstatClient
uc = UnderstatClient()

YEAR = '2019'

understat = UnderstatClient()
# get data for every player playing in the Premier League in 2019/20

league_player_ids = set()
for oneyear in ['2019']:
    league_player_data = understat.league(league="EPL").get_player_data(season=oneyear)
    league_player_ids.update( [player['id'] for player in league_player_data])

In [2]:
def process_shot_data(player_shot_data):
    for shot in player_shot_data:
        shot['ScoredEarlier']=0
        shot['ScoredPenEarlier']=0
        shot['ScoredLater']=0
        shot['ScoredPenLater']=0

        # if shot['a_goals']==shot['h_goals']:
        #     shot['State']='tied'
        # elif (shot['a_goals']>shot['h_goals'] and shot['h_a']=='a') or (shot['a_goals']<shot['h_goals'] and shot['h_a']=='h'):
        #     shot['State']='winning'
        # else:
        #     shot['State']='losing'
        # ovo ne radi jer H_GOALS i A_GOALS prikazuju rezultat na kraju, a ne u trenutku suta.
        
        if(shot['result']=='Goal'):
            shot['IsGoal']=True
        else:
            shot['IsGoal']=False


    for i in range(len(player_shot_data)):
        shot = player_shot_data[i]
        original_date = (shot['date'].split()[0])
        if(shot['result']=='Goal'):
            j=i+1
            while  j<len(player_shot_data) and player_shot_data[j]['date'].split()[0] == original_date:
                player_shot_data[j]['ScoredEarlier']=1
                j+=1
            
            
            j=i-1
            while player_shot_data[j]['date'].split()[0] == original_date and j>=0:
                player_shot_data[j]['ScoredLater']=1
                j-=1
            
            
        # print(shot['date'].split()[0])
        # print(shot['minute'])




In [3]:

all_player_shot_data = []
for player_id in league_player_ids:
    shot_data = understat.player(player=player_id).get_shot_data()
    process_shot_data(shot_data)
    all_player_shot_data+=shot_data





In [4]:

from heapq import merge


match_shots = [understat.match(match_info['id']).get_shot_data() for match_info in understat.league('EPL').get_match_data(season = YEAR)]

all_match_shot_data = []

for i in range(len(match_shots)):
    team_dic = match_shots[i]
    match=list(merge(team_dic['a'], team_dic['h'], key=lambda shot : int(shot['minute'])))

    home_goals_so_far=0
    away_goals_so_far=0
    for shot in match:
        if shot['h_a']=='h':
            shot['MyScore']=home_goals_so_far
            shot['OppScore']=away_goals_so_far
        else:
            shot['MyScore']=away_goals_so_far
            shot['OppScore']=home_goals_so_far

        if shot['result']=='Goal':
            if shot['h_a']=='h':
                home_goals_so_far+=1
            else:
                away_goals_so_far+=1
    match_shots[i]=match
    all_match_shot_data+=match




In [5]:
matchshotdf = pd.DataFrame(all_match_shot_data).set_index('id')
matchshotdf.head(10)

,minute,result,X,Y,xG,player,h_a,player_id,situation,season,...,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction,MyScore,OppScore
id,,,,,,,,,,,,,,,,,,,,,
309892,4,MissedShots,0.8690000152587891,0.629000015258789,0.07037080079317093,Marco Stiepermann,a,7694,OpenPlay,2019,...,11643,Liverpool,Norwich,4,1,2019-08-09 20:00:00,Teemu Pukki,Pass,0,0
309893,6,OwnGoal,0.035999999046325684,0.4179999923706055,0,Grant Hanley,a,7690,OpenPlay,2019,...,11643,Liverpool,Norwich,4,1,2019-08-09 20:00:00,None,None,0,0
309894,9,MissedShots,0.84,0.6140000152587891,0.04457490146160126,Marco Stiepermann,a,7694,OpenPlay,2019,...,11643,Liverpool,Norwich,4,1,2019-08-09 20:00:00,Teemu Pukki,Pass,0,0
309895,14,BlockedShot,0.895,0.6780000305175782,0.09858030080795288,Teemu Pukki,a,7696,OpenPlay,2019,...,11643,Liverpool,Norwich,4,1,2019-08-09 20:00:00,Jamal Lewis,Throughball,0,0
309896,14,MissedShots,0.97,0.6719999694824219,0.05384039878845215,Teemu Pukki,a,7696,OpenPlay,2019,...,11643,Liverpool,Norwich,4,1,2019-08-09 20:00:00,Todd Cantwell,Pass,0,0
309897,15,MissedShots,0.8830000305175781,0.7130000305175781,0.053779199719429016,Andrew Robertson,h,1688,OpenPlay,2019,...,11643,Liverpool,Norwich,4,1,2019-08-09 20:00:00,Divock Origi,Pass,0,0
309898,18,Goal,0.9330000305175781,0.36700000762939455,0.36681699752807617,Mohamed Salah,h,1250,OpenPlay,2019,...,11643,Liverpool,Norwich,4,1,2019-08-09 20:00:00,Roberto Firmino,Pass,0,0
309899,24,SavedShot,0.8059999847412109,0.4779999923706055,0.04139560088515282,Emiliano Buendía,a,2203,OpenPlay,2019,...,11643,Liverpool,Norwich,4,1,2019-08-09 20:00:00,Todd Cantwell,Pass,0,1
309900,25,SavedShot,0.8030000305175782,0.49,0.05499729886651039,Marco Stiepermann,a,7694,OpenPlay,2019,...,11643,Liverpool,Norwich,4,1,2019-08-09 20:00:00,Emiliano Buendía,Pass,0,1


In [6]:
shotdf = pd.DataFrame(all_player_shot_data)
shotdf = shotdf[ shotdf['season'] == YEAR ].set_index('id')

print(shotdf.head(10),
matchshotdf[['MyScore','OppScore']].head(10))

shotdf = shotdf.join(matchshotdf[['MyScore','OppScore']], how='inner')

shotdf['minute'] = shotdf['minute'].apply(int)
shotdf['xG'] = shotdf['xG'].apply(float)



       minute       result                   X                    Y  \
id                                                                    
310300     31  MissedShots                0.92                0.435   
311303     67    SavedShot               0.885                  0.5   
311305     70  MissedShots  0.9019999694824219   0.5509999847412109   
312587     41  MissedShots  0.7980000305175782   0.6079999923706054   
312591     52  BlockedShot  0.8230000305175781    0.539000015258789   
312598     76  MissedShots   0.919000015258789  0.48400001525878905   
312600     78  MissedShots               0.895  0.48200000762939454   
323711     36  BlockedShot   0.735999984741211  0.35200000762939454   
323724     63  MissedShots  0.7809999847412109    0.610999984741211   
339207     75    SavedShot   0.735999984741211    0.779000015258789   

                          xG      player h_a player_id       situation season  \
id                                                                

In [7]:
totalxg=shotdf['xG'].sum()
totalgoals=shotdf['IsGoal'].sum()
print(totalgoals / totalxg)
print(len(shotdf[shotdf['ScoredLater']==1]['IsGoal'].index))
print( shotdf[shotdf['ScoredLater']==1]['IsGoal'].sum() / shotdf[ shotdf['ScoredLater'] == 1]['xG'].sum())

0.9114096440776038
935
0.8869362928593179


In [8]:
print(len(shotdf.index))
shotdf = shotdf[shotdf['xG']>0.04]
print(len(shotdf.index))
shotdf['ScaledIsGoal'] = shotdf['IsGoal'] / shotdf['xG']
shotdf['ScaledIsGoal'].head(30)

9427
6007


id
310300    0.000000
311303    0.000000
312587    0.000000
312591    0.000000
312598    0.000000
339210    0.000000
339664    0.000000
339670    0.000000
367766    0.000000
368575    0.000000
310069    0.000000
309989    0.000000
352946    0.000000
367624    0.000000
369435    0.000000
373706    0.000000
330425    0.000000
337950    0.000000
352138    0.000000
368598    0.000000
352338    0.000000
352566    0.000000
365871    0.000000
371645    0.000000
312528    0.000000
317342    0.000000
317344    3.997665
332341    0.000000
332343    0.000000
336306    0.000000
Name: ScaledIsGoal, dtype: float64

In [9]:
shotdf.keys()

Index(['minute', 'result', 'X', 'Y', 'xG', 'player', 'h_a', 'player_id',
       'situation', 'season', 'shotType', 'match_id', 'h_team', 'a_team',
       'h_goals', 'a_goals', 'date', 'player_assisted', 'lastAction',
       'ScoredEarlier', 'ScoredPenEarlier', 'ScoredLater', 'ScoredPenLater',
       'IsGoal', 'MyScore', 'OppScore', 'ScaledIsGoal'],
      dtype='object')

In [10]:
# %pip install scikit-learn
from sklearn.linear_model import LinearRegression

shotdf['IsHome'] = (shotdf['h_a']=='h')
#shotdf[shotdf['h_a']=='h']['IsHome'] = 1
X = shotdf[['minute','IsHome','ScoredEarlier','MyScore','OppScore']]
y = shotdf['ScaledIsGoal']

model = LinearRegression()
model.fit(X, y)
print(model.coef_, model.intercept_)

LinearRegression()


<class 'numpy.ndarray'>


In [61]:
import json
betting_nameset = set()
def get_season_betting_data(year, translator=None):
    betting_nameset = set()
    jsondata=None
    with open(f'../data/{year}.json') as f:
        jsondata = json.load(f)
    print( jsondata['d']['rows'][0].keys() )

    print( jsondata['d']['rows'][0]['odds'][2] ) #proveriti da li je 3.58 za nearseal tim.
    raise Exception
    print( jsondata['d']['rows'][0]['date-start-timestamp'] )
    if translator is None:
        for row in jsondata['d']['rows']:
            betting_nameset.add(row['home-name'])
        return betting_nameset, None
    else:
        for row in jsondata['d']['rows']:
            match_id = (year, row['home-name'], row['away-name'])
            # shotdf.loc[shotdf[['season','h_team','a_team']]==pd.Series(match_id) , 'HomeOdds'] = row['odds'][0]['avgOdds']  - da li ovo radi?


#ovaj deo sluzi samo za nameset! kasnije se zap koristi get_season_betting_data

names, _ = get_season_betting_data(2019)
print(names)
betting_nameset = betting_nameset.union(names)
names, _ = get_season_betting_data(2020)
etting_nameset = betting_nameset.union(names)

names, _ = get_season_betting_data(2021)
betting_nameset = betting_nameset.union(names)

names, _ = get_season_betting_data(2022)
betting_nameset = betting_nameset.union(names)
print(betting_nameset)

dict_keys(['id', 'is-double', 'superTemplate', 'home', 'away', 'home-name', 'away-name', 'home-country-two-chart-name', 'away-country-two-chart-name', 'home-participant-id', 'away-participant-id', 'status-id', 'event-stage-id', 'event-stage-name', 'event-stage-name-short', 'tournament_id', 'tournament-stage-id', 'tournament-stage-type-id', 'tournament-stage-group-id', 'tournament-stage-name', 'sport-id', 'cols', 'home-participant-images', 'away-participant-images', 'encodeEventId', 'colClassName', 'homeParticipantTypes', 'awayParticipantTypes', 'ftOnly', 'ftOnlyText', 'ftOnlyTextShort', 'url', 'date-start-base', 'date-start-timestamp', 'result', 'homeResult', 'awayResult', 'homeTextInResult', 'awayTextInResult', 'home-winner', 'away-winner', 'postmatchResult', 'info', 'partialresult', 'country-id', 'country-name', 'country-two-chart-name', 'country-type', 'tournament-id', 'tournament-name', 'tournament-url', 'sport-url-name', 'breadcrumbs', 'stream', 'bookmakersCount', 'winner_post', '

Exception: 

In [45]:
understat_nameset = set()
shotdf.apply(axis = 1, func = lambda row : understat_nameset.add(row['h_team']))
print(understat_nameset)

beting_iter = iter(betting_nameset)

{'Aston Villa', 'West Ham', 'Manchester City', 'Crystal Palace', 'Arsenal', 'Burnley', 'Everton', 'Liverpool', 'Bournemouth', 'Wolverhampton Wanderers', 'Newcastle United', 'Brighton', 'Tottenham', 'Southampton', 'Manchester United', 'Chelsea', 'Norwich', 'Sheffield United', 'Watford', 'Leicester'}


In [57]:
name_translator = {}
l_betting_nameset = list(betting_nameset)
for name in l_betting_nameset:
    if name in understat_nameset:
        name_translator[name]=name
        understat_nameset.remove(name)
        betting_nameset.remove(name)
print(betting_nameset)
name_translator['Manchester Utd'] = 'Manchester United'
name_translator['Wolves'] = 'Wolverhampton Wanderers'
name_translator['Newcastle'] = 'Newcastle United'
name_translator['Sheffield Utd'] = 'Sheffield United'
name_translator['Nottingham'] = 'Nottingham Forest'
name_translator['Brentford'] = 'Brentford'
name_translator['Fulham'] = 'Fulham'
name_translator['Leeds'] = 'Leeds'


{'Brentford', 'Fulham', 'Manchester Utd', 'Newcastle', 'Leeds', 'Sheffield Utd', 'Wolves', 'Nottingham'}
{'Wolverhampton Wanderers', 'Newcastle United', 'Manchester United', 'Sheffield United'}


# Zastareo kod

In [12]:
xg_if_scored_earlier = 0
xg_if_scored_later = 0
total_xg = 0
scored_if_scored_earlier = 0
scored_if_scored_later = 0
total_shots = len(all_player_shot_data)
total_goals = 0
for shot in all_player_shot_data:
    if shot['IsGoal']:
        total_goals+=1
        if shot['ScoredEarlier']==1:
            scored_if_scored_earlier+=1
        if shot['ScoredLater']==1:
            scored_if_scored_later+=1
    if shot['ScoredEarlier']==1:
        xg_if_scored_earlier+=float(shot['xG'])
    if shot['ScoredLater']==1:
        xg_if_scored_later+=float(shot['xG'])
    total_xg += float(shot['xG'])

print(scored_if_scored_earlier, scored_if_scored_later)
print(xg_if_scored_earlier, xg_if_scored_later)
ratio_if_scored_earlier = scored_if_scored_earlier / xg_if_scored_earlier
ratio_if_scored_later = scored_if_scored_later / xg_if_scored_later
ratio_total = total_goals / total_xg
print(f'{ratio_if_scored_earlier=} {ratio_if_scored_later=} {ratio_total=}')

930 930
1015.6971548674628 1049.4685114114545
ratio_if_scored_earlier=0.9156272571437445 ratio_if_scored_later=0.8861628432750416 ratio_total=0.9748580611219628


In [13]:
# print([ lis['id'] for lis in (match_dic[('2015-02-07 15:00:00', 'Leicester', 'Crystal Palace')]) ])
# print([ lis['id'] for lis in (match_dic[('2014-10-04 15:00:00', 'Leicester', 'Burnley')]) ] )

# 'h_team': 'Leicester', 'a_team': 'Burnley', 'h_goals': '2', 'a_goals': '2', 'date': '2014-10-04 15:00:00'


In [14]:
from functools import reduce
def get_xg_and_goals_for_condition_combination(shot_data, condition_dic):
    keys = list(condition_dic.keys())
    vals = list(condition_dic.values())

    conditional_xg = 0
    conditional_goals = 0

    for shot in shot_data:
        # if len(args)==0 or reduce((lambda x, y: x and y), [shot[arg] for arg in args]):
        if vals == [shot[arg] for arg in keys]:
            conditional_xg += float(shot['xG'])
            conditional_goals += int(shot['IsGoal'])
    return conditional_xg, conditional_goals
print(get_xg_and_goals_for_condition_combination(all_player_shot_data, {'ScoredEarlier': True}) )

# 'State' nije dobar. otkomentarisi ovo kad iskoristis match data da 

# print(get_xg_and_goals_for_condition_combination(all_player_shot_data, {'ScoredEarlier': True ,'State': 'winning'}))
# print(get_xg_and_goals_for_condition_combination(all_player_shot_data, {'ScoredEarlier': True ,'State': 'tied'}))
# print(get_xg_and_goals_for_condition_combination(all_player_shot_data, {'ScoredEarlier': True ,'State': 'losing'}))
# print(get_xg_and_goals_for_condition_combination(all_player_shot_data, {'State': 'winning'}))
# print(get_xg_and_goals_for_condition_combination(all_player_shot_data, {'State': 'tied'}))
# print(get_xg_and_goals_for_condition_combination(all_player_shot_data, {'State': 'losing'}))

(1015.6971548674628, 930)


In [15]:
[1,2]==[True,2]

True

In [16]:
print(all_player_shot_data[0])

{'id': '41441', 'minute': '3', 'result': 'BlockedShot', 'X': '0.735', 'Y': '0.55', 'xG': '0.01573289930820465', 'player': 'Paul Pogba', 'h_a': 'a', 'player_id': '1740', 'situation': 'OpenPlay', 'season': '2014', 'shotType': 'RightFoot', 'match_id': '5149', 'h_team': 'Chievo', 'a_team': 'Juventus', 'h_goals': '0', 'a_goals': '1', 'date': '2014-08-30 17:00:00', 'player_assisted': None, 'lastAction': 'None', 'ScoredEarlier': 0, 'ScoredPenEarlier': 0, 'ScoredLater': 0, 'ScoredPenLater': 0, 'IsGoal': False}


# Treba da se odradi game state, znaci da li je u trenutku suta taj tim gubio, pobedjivao ili su bili izjednaceni.
Treba u recniku match_data napraviti hronoloski sortiranu listu suteva (koristeci id) koji su ispali golovi.  onda kad se to zavrsi da se vratimo u shot data i da napravimo jos jednu kolonu / key. ovo ovako mora zato sto je shot data prvenstveno sortiran po igracu pa tek onda po vremenu. Treba koristiti match_id i shot_id sto vise, i cinjenicu da je 0-10 puta u istoriji (jako retko lupam brojeve) dato vise golova u istom minutu (sto je dovoljno da ne utice na podatke)

In [17]:
understat.match('22009').get_shot_data()



{'h': [{'id': '549079',
   'minute': '0',
   'result': 'BlockedShot',
   'X': '0.8590000152587891',
   'Y': '0.2879999923706055',
   'xG': '0.038173574954271317',
   'player': 'Reece James',
   'h_a': 'h',
   'player_id': '8067',
   'situation': 'OpenPlay',
   'season': '2023',
   'shotType': 'RightFoot',
   'match_id': '22009',
   'h_team': 'Chelsea',
   'a_team': 'Manchester City',
   'h_goals': '4',
   'a_goals': '4',
   'date': '2023-11-12 16:30:00',
   'player_assisted': 'Cole Palmer',
   'lastAction': 'BallTouch'},
  {'id': '549080',
   'minute': '11',
   'result': 'BlockedShot',
   'X': '0.8909999847412109',
   'Y': '0.3129999923706055',
   'xG': '0.04885943979024887',
   'player': 'Cole Palmer',
   'h_a': 'h',
   'player_id': '8497',
   'situation': 'OpenPlay',
   'season': '2023',
   'shotType': 'LeftFoot',
   'match_id': '22009',
   'h_team': 'Chelsea',
   'a_team': 'Manchester City',
   'h_goals': '4',
   'a_goals': '4',
   'date': '2023-11-12 16:30:00',
   'player_assisted'

In [18]:

# match_dic = {}
# def create_match_dic(row):
#     match_dic[(row['date'], row['h_team'], row['a_team'])] = match_dic.get((row['date'], row['h_team'], row['a_team']),[]) + [row]

# shotdf.apply( func = create_match_dic, axis=0)

# print(shotdf.head(10))

# print(list(match_dic.values())[:10])
# for val in match_dic.values():
#     val.sort(key = lambda row : row['minute'])





In [19]:
(shotdf[shotdf['date']=='2019-08-24 15:00:00'] ).sort_values(by=['h_team','minute']).to_csv('this_date.csv')